In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 144.6MB 74kB/s 
     |████████████████████████████████| 61kB 2.9MB/s 
ERROR: earthengine-api 0.1.258 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  39053      0 --:--:-- --:--:-- --:--:-- 39053
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Copying gs://tpu-pytorch/wheels/torch-nightly-cp37-cp37m-linux_x86_64.whl...
\ [1 files][131.8 MiB/131.8 MiB]                                                
Operation completed over 1 objects/131.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp37-cp37m-linux_x86_64.whl...
- [1 files][138.4 MiB/138.4 MiB]                                                
Operation completed over 1 objects/138.4 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly-cp37-cp37m-linux_x86_64.whl...
/ [1 files][  4.9 MiB/  4.9 MiB]                                                
Operation completed over 1 

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 5.2MB/s 
     |████████████████████████████████| 870kB 20.3MB/s 
     |████████████████████████████████| 3.3MB 42.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=50682b0a7730eb0b8db5af63f585dcbc086689b63ab66cf146d075dc958e161a
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import os

os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'
import random
import copy
import spacy
# import unidecode
import glob
import numpy as np
import torch
import pandas as pd
from google.colab import drive
from tqdm.notebook import tqdm
import nltk
from torch.utils.data import Dataset, random_split
from torch.utils.data.distributed import DistributedSampler
import re
# from spellchecker import SpellChecker
import string
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from collections import Counter, defaultdict
import torch.nn as nn
from sklearn import utils
import gensim.parsing.preprocessing as gsp
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
# from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import utils
import gensim.parsing.preprocessing as gsp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_union
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_union
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torchtext import data, vocab, legacy
import torch.optim as optim
import logging
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,BertPreTrainedModel
)
from collections import deque
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import time

In [ ]:
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([0,1])
        self.y_pred = np.array([0.5,0.5])
        self.score = 0

    def update(self, y_true, y_pred):
        # y_true = y_true.cpu().numpy().argmax(axis=1)
        y_true = y_true.cpu().numpy()
        y_pred = nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = metrics.roc_auc_score(self.y_true, self.y_pred, labels=np.array([0, 1]))
    
    @property
    def avg(self):
        return self.score

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def clean(text):
  if re.findall(r'\[\'\'\]',text):
    return ''
  else:
    retext = re.split(',',text)
    clean_text = []
    retext[0] = retext[0][1:]
    retext[-1] = retext[-1][:-1]
    for i in range(len(retext)):
        retext[i] = " ".join(retext[i].split())
        retext[i] = retext[i][1:-1]
    return '.'.join(retext)

def clean_test(text):
  if re.findall(r'\[\'\'\]',text):
    return '[UNK]'
  else:
    retext = re.split(',',text)
    clean_text = []
    retext[0] = retext[0][1:]
    retext[-1] = retext[-1][:-1]
    for i in range(len(retext)):
        retext[i] = " ".join(retext[i].split())
        retext[i] = retext[i][1:-1]
    return '.'.join(retext)

MAX_SEQ_LENGTH = 200

In [ ]:
MAX_LEN=200
# class TextDataset(torch.utils.data.TensorDataset):

#     def __init__(self, data, idxs, targets=None):
#         self.input_ids = data[0][idxs]
#         self.input_masks = data[1][idxs]
#         self.input_segments = data[2][idxs]
#         self.targets = targets[idxs] if targets is not None else np.zeros((data[0].shape[0], 1))
#     def __getitem__(self, idx):
#         input_ids =  self.input_ids[idx]
#         input_masks = self.input_masks[idx]
#         input_segments = self.input_segments[idx]

#         target = self.targets[idx]

#         return input_ids, input_masks, input_segments, target

#     def __len__(self):
#         return len(self.input_ids)

class TextDataset(torch.utils.data.TensorDataset):

  def __init__(self, comment_text, targets, max_length, tokenizer, test=False):
    self.comment_text = comment_text
    self.test = test
    self.tokenizer = tokenizer
    self.max_length = max_length
    self.targets = targets

  def get_tokens(self, text):
    encoded = tokenizer.encode_plus(
        text, 
        add_special_tokens=True, 
        max_length=self.max_length, 
        pad_to_max_length=True
    )
    return encoded['input_ids'], encoded['attention_mask']

  def __getitem__(self, item):
    
    text = self.comment_text[item]
            
    encoded = self.get_tokens(text)
    
    if not self.test:
      return torch.tensor(encoded[0],dtype=torch.long), torch.tensor(encoded[1],dtype=torch.long), torch.tensor(self.targets[item], dtype=torch.float)
    else:
      return torch.tensor(encoded[0],dtype=torch.long), torch.tensor(encoded[1])          

  def __len__(self):
      return len(self.targets)

In [ ]:
TextDataset(train.comment_text.values,train.toxic.values)

NameError: ignored

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
# def get_bert_tokenize(df):
    
#     token_ids, segment_ids, attention_mask = np.zeros((len(df), MAX_SEQ_LENGTH)), np.zeros((len(df), MAX_SEQ_LENGTH)), np.zeros((len(df), MAX_SEQ_LENGTH))
    
#     for i, content in tqdm(enumerate(df.comment_text.values), total=len(df)):
        
#         content = tokenizer.tokenize(content)[:MAX_SEQ_LENGTH-2]
#         token_id = tokenizer.encode(content)
        
#         token_ids[i] = token_id + [0] * (MAX_SEQ_LENGTH-len(token_id))
#         segment_ids[i] = [0] * MAX_SEQ_LENGTH               
#         attention_mask[i] = [1] * len(token_id) + [0] * (MAX_SEQ_LENGTH - len(token_id))
        
#     return token_ids, segment_ids, attention_mask

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Epam/train_sent_prep.csv',usecols=["comment_text", "toxic"],
                    converters={'comment_text': lambda x: clean(x)})
# train = pd.read_csv('/content/drive/My Drive/epam/2.0/Module 2/train_sent_prep.csv')

valid = pd.read_csv('/content/drive/My Drive/Epam/validation_sent_pr.csv', 
                    usecols=["comment_text", "toxic"],
                    converters={'comment_text': lambda x: clean(x)})

test = pd.read_csv('/content/drive/My Drive/Epam/test_sent_pr.csv', usecols=["correct_text"],
                    converters={'correct_text': lambda x: clean_test(x)})

test.comment_text=test.correct_text

# %%time
# x_train = get_bert_tokenize(train[train.comment_text!=''])
# x_test = get_bert_tokenize(test)
# x_validation = get_bert_tokenize(valid[valid.comment_text!=''])

In [ ]:
# bert_config = BertConfig.from_pretrained('bert-base-multilingual-uncased') 

In [ ]:
# import pickle
# with open('x_train.pickle','wb') as f:
#   pickle.dump(x_train,f)
# with open('x_test.pickle','wb') as f:
#   pickle.dump(x_test,f)
# with open('x_validation.pickle','wb') as f:
#   pickle.dump(x_validation,f)

In [ ]:
# import pickle
# with open('/content/drive/MyDrive/Epam/x_train.pickle','rb') as f:
#   x_train = pickle.load(f)
# with open('/content/drive/MyDrive/Epam/x_test.pickle','rb') as f:
#   x_test = pickle.load(f)
# with open('/content/drive/MyDrive/Epam/x_validation.pickle','rb') as f:
#   x_validation = pickle.load(f)

In [ ]:
# class CustomizedBert(BertPreTrainedModel):
  
#     def __init__(self, config):
#         super(CustomizedBert, self).__init__(config)
#         self.num_labels = config.num_labels

#         self.bert = BertModel(config)
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.classifier = nn.Linear(config.hidden_size*2, self.config.num_labels)

#         self.init_weights()
#     def forward(
#         self,
#         input_ids=None,
#         attention_mask=None,
#         token_type_ids=None,
#         position_ids=None,
#         head_mask=None,
#         inputs_embeds=None,
#         labels=None,
#     ):

#         outputs = self.bert(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             position_ids=position_ids,
#             head_mask=head_mask,
#             inputs_embeds=inputs_embeds,
#         )

#         ## mean max pooling and concatenate to a vector
        
#         avg_pool = torch.mean(outputs[0], 1)
#         max_pool, _ = torch.max(outputs[0], 1)
#         pooled_output = torch.cat((max_pool, avg_pool), 1)

#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)

#         outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

#         return outputs  # (loss), logits, (hidden_states), (attentions)


# class callback:
#     def __init__(self):
#         self.score = list()
#         self.model = list()
    
#     def put(self, model, score):
#         self.score.append(score)
#         self.model.append(model)

#     def get_model(self):
#         ind = np.argmin(self.score)
#         return self.model[ind]
    

# def model_validation(net, val_loader, validation_length):
    
#     avg_val_loss = 0.0
    
#     net.eval()
#     valid_preds = np.zeros((validation_length, 1))
#     true_label = np.zeros((validation_length, 1))
    
#     for j, data in enumerate(val_loader):

#         # get the inputs
#         input_ids, input_masks, input_segments, labels = data
#         pred = net(input_ids = input_ids.long().to(device),
#                          labels = None,
#                          attention_mask = input_masks.to(device),
#                          token_type_ids = input_segments.long().to(device)
#                         )[0]

#         loss_val = loss_fn(pred, labels.float().to(device))
#         avg_val_loss += loss_val.item()

#         valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = torch.sigmoid(pred).cpu().detach().numpy()
#         true_label[j * BATCH_SIZE:(j+1) * BATCH_SIZE]  = labels.float()


#     score = roc_auc_score(true_label, valid_preds)
    
#     return valid_preds, avg_val_loss, score

train = train[train.comment_text!=''].reset_index(drop=True)
valid = valid[valid.comment_text!=''].reset_index(drop=True)

In [ ]:
def loss_fn(outputs, targets):
    # pass in outputs and targets, return loss function
    return torch.nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

In [ ]:
def train_loop_fn(loader,model,device):
  tracker = xm.RateTracker()
  # cb = callback()
  losses = AverageMeter()
  auc = RocAucMeter()
  # start_time = time.time()


  gradient_accumulation_steps = 1
  # avg_loss = 0.0
  model.train()
  for step, data in enumerate(tqdm(loader)):
      input_ids, input_masks, labels = data
      pred = model(input_ids = input_ids,
                    attention_masks = input_masks
                  )
      # train_preds.extend(torch.sigmoid(pred).cpu().detach().numpy())
      # train_labels.extend(labels.float())
      loss = loss_fn(pred, labels.float())
      # avg_loss += loss.item()
      # loss = loss / gradient_accumulation_steps
      loss.backward()
      loss = loss.detach().item()
        
      auc.update(labels, pred)
      losses.update(loss, input_ids.size(0))
      if (step + 1) % gradient_accumulation_steps == 0:
          # Calling the step function on an Optimizer makes an update to its parameters
          xm.optimizer_step(optimizer)
          scheduler.step()
          optimizer.zero_grad()

      tracker.add(FLAGS['batch_size'])
      if (step+1) % FLAGS['log_steps'] == 0:
          print('[xla:{}]({}) Rate={:.2f} Loss={:.4f} AUC={:.4f} GlobalRate={:.2f} Time={}'.format(
              xm.get_ordinal(), step+1, tracker.rate(), losses.avg, auc.avg,
              tracker.global_rate(), time.asctime()), flush=True)
  del loss
  del losses
  del outputs
  del input_ids
  del labels
  
  return losses, auc

def valid_loop_fn(loader,model,device): 
  fin_targets = []
  fin_outputs = []   
  losses = AverageMeter()
  final_scores = RocAucMeter()

  model.eval()
  # valid_preds = np.zeros((validation_length, 1))
  # true_label = np.zeros((validation_length, 1))
  with torch.no_grad():
    for j, data in enumerate(val_loader):

        # get the inputs
        input_ids, input_masks, labels = data
        pred = net(input_ids = input_ids.long(),
                          attention_mask = input_masks
                        )

        loss_val = loss_fn(pred, labels.float())
        final_scores.update(targets, outputs)
        losses.update(loss.detach().item(), input_ids.size(0))
        del loss_val,pred,input_ids,input_masks,labels

  return losses, final_scores

In [ ]:
FLAGS = {}
FLAGS['seed'] = 1
FLAGS['batch_size'] = 32
FLAGS['num_workers'] = 0
FLAGS['num_epochs'] = 5
FLAGS['num_cores'] = 8
FLAGS['log_steps'] = 20

In [ ]:
# import os
# torch.manual_seed(FLAGS['seed'])
# device = xm.xla_device()
# gradient_accumulation_steps = 1
# seed_everything(FLAGS['seed'])
# lr = 3e-5
# model_list = list()

# y_train = train.toxic.values.reshape(-1, 1)
# y_val = valid.toxic.values.reshape(-1, 1)

# test_pred = np.zeros((len(test), 1))

# test_loader = torch.utils.data.DataLoader(TextDataset(x_test, test.index),batch_size=FLAGS['batch_size'], shuffle=False)
# # train_dataset = TextDataset(x_train, train.index, y_train)
# # valid_dataset = TextDataset(x_validation, valid.index, y_val)

# # train_sampler = DistributedSampler(train_dataset,
# #                                     num_replicas = xm.xrt_world_size(),
# #                                     rank = xm.get_ordinal(),
# #                                     shuffle = True)
# # valid_sampler = DistributedSampler(valid_dataset,
# #                                     num_replicas = xm.xrt_world_size(),
# #                                     rank = xm.get_ordinal(),
# #                                     shuffle = True)    
# ## loader
# train_loader = torch.utils.data.DataLoader(TextDataset(x_train, train.index, y_train),
#                                             batch_size=FLAGS['batch_size'], shuffle=True)
#                                             # num_workers = FLAGS['num_workers']
#                                             # sampler=train_sampler)
# val_loader = torch.utils.data.DataLoader(TextDataset(x_validation, valid.index, y_val),
#                                           batch_size=FLAGS['batch_size'], shuffle=False)
#                                           # num_workers = FLAGS['num_workers']
#                                         #  sampler=valid_sampler, 
# # print('created loaders')
# t_total = len(train_loader)//gradient_accumulation_steps*FLAGS['num_epochs']
# warmup_proportion = 0.01
# num_warmup_steps = t_total * warmup_proportion

# net = CustomizedBert.from_pretrained('bert-base-multilingual-uncased', config=bert_config)
# net = net.to(device)
# # print('Объявил модель')
# # loss_window = deque(maxlen = FLAGS['log_steps'])
# loss_fn = torch.nn.BCEWithLogitsLoss().to(device)
# optimizer = AdamW(net.parameters(), lr = lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)  # PyTorch scheduler
# best_valid_loss = float("Inf")

In [ ]:
# cb = callback()

# SEED = 2020
# lr = 3e-5


# gradient_accumulation_steps = 1
# seed_everything(SEED)
# avg_loss = 0.0
# # train_preds = np.zeros((len(train), 1))
# # train_labels = np.zeros((len(train), 1))
# # train_preds = []
# # train_labels = []
# net.train()
# for step, data in enumerate(tqdm(train_loader)):
#     input_ids, input_masks, input_segments, labels = data
#     pred = net(input_ids = input_ids.long().to(device),
#                   labels = None,
#                   attention_mask = input_masks.to(device),
#                   token_type_ids = input_segments.long().to(device)
#                 )[0]
#     # train_preds.extend(torch.sigmoid(pred.cpu().detach()).numpy())
#     # train_labels.extend(labels.float())
#     loss = loss_fn(pred, labels.float().to(device))
#     avg_loss += loss.item()
#     loss = loss / gradient_accumulation_steps
#     loss.backward()
#     # train_preds[step * FLAGS['batch_size']:(step+1) * FLAGS['batch_size']] = torch.sigmoid(pred).cpu().detach().numpy()
#     # train_labels[step * FLAGS['batch_size']:(step+1) * FLAGS['batch_size']]  = labels.float()
    
#     if (step + 1) % gradient_accumulation_steps == 0:
#         # Calling the step function on an Optimizer makes an update to its parameters
#         xm.optimizer_step(optimizer, barrier=True)
#         scheduler.step()
#         optimizer.zero_grad()

#     # tracker.add(FLAGS['batch_size'])
#     # if (step+1) % FLAGS['log_steps'] == 0:
#     #     print('[xla:{}]({}) Rate={:.2f} GlobalRate={:.2f} Time={}'.format(
#     #         xm.get_ordinal(), x+1, tracker.rate(),
#     #         tracker.global_rate(), time.asctime()), flush=True)
# score = roc_auc_score(train_labels, train_preds)
# return avg_loss/len(loader), score

In [ ]:
# for step, data in enumerate(tqdm(train_loader)):
#     input_ids, input_masks, input_segments, labels = data
#     pred = net(input_ids = input_ids.long().to(device),
#                   labels = None,
#                   attention_mask = input_masks.to(device),
#                   token_type_ids = input_segments.long().to(device)
#                 )[0]
#     # train_preds.extend(torch.sigmoid(pred.cpu().detach()).numpy())
#     # train_labels.extend(labels.float())
#     loss = loss_fn(pred, labels.float().to(device))
#     avg_loss += loss.item()
#     loss = loss / gradient_accumulation_steps
#     loss.backward()
#     # train_preds[step * FLAGS['batch_size']:(step+1) * FLAGS['batch_size']] = torch.sigmoid(pred).cpu().detach().numpy()
#     # train_labels[step * FLAGS['batch_size']:(step+1) * FLAGS['batch_size']]  = labels.float()
    
#     if (step + 1) % gradient_accumulation_steps == 0:
#         # Calling the step function on an Optimizer makes an update to its parameters
#         xm.optimizer_step(optimizer, barrier=True)
#         scheduler.step()
#         optimizer.zero_grad()

#     # tracker.add(FLAGS['batch_size'])
#     # if (step+1) % FLAGS['log_steps'] == 0:
#     #     print('[xla:{}]({}) Rate={:.2f} GlobalRate={:.2f} Time={}'.format(
#     #         xm.get_ordinal(), x+1, tracker.rate(),
#     #         tracker.global_rate(), time.asctime()), flush=True)
# score = roc_auc_score(train_labels, train_preds)
# return avg_loss/len(loader), score

In [ ]:
net = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased')

KeyboardInterrupt: ignored

In [ ]:
class ToxicSimpleNNModel(nn.Module):

    def __init__(self):
        super(ToxicSimpleNNModel, self).__init__()
        self.encoder = BertModel.from_pretrained('bert-base-multilingual-uncased')
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(
            in_features=self.encoder.pooler.dense.out_features*2,
            out_features=1,
        )

    def forward(self, input_ids, attention_masks):
        # bs, seq_length = input_ids.shape
        seq_x = self.encoder(input_ids=input_ids, attention_mask=attention_masks)
        apool = torch.mean(seq_x[0], 1)
        mpool, _ = torch.max(seq_x[0], 1)
        x = torch.cat((apool, mpool), 1)
        x = self.dropout(x)
        return self.linear(x)

In [ ]:
MX = xmp.MpModelWrapper(ToxicSimpleNNModel())

In [ ]:
train = train[:60000]

In [ ]:
mx = ToxicSimpleNNModel()

In [ ]:
import os
def train_net(FLAGS):
  torch.manual_seed(FLAGS['seed'])
  seed_everything(FLAGS['seed'])
  lr = 3e-5
  model_list = list()

  # y_train = train.toxic.values.reshape(-1, 1)
  # y_val = valid.toxic.values.reshape(-1, 1)

  # test_pred = np.zeros((len(test), 1))
  count_proc = xm.xrt_world_size()
  num_proc = xm.get_ordinal()
  size_tr_dataset = round(len(train)/count_proc)
  size_val_dataset = round(len(valid)/count_proc)
  gradient_accumulation_steps=1

  # test_loader = torch.utils.data.DataLoader(TextDataset(x_test, test.index),batch_size=FLAGS['batch_size'], shuffle=False)
  # train_dataset = TextDataset(x_train, train.index, y_train)
  # valid_dataset = TextDataset(x_validation, valid.index, y_val)

  # train_sampler = DistributedSampler(train_dataset,
  #                                     num_replicas = xm.xrt_world_size(),
  #                                     rank = xm.get_ordinal(),
  #                                     shuffle = True)
  # valid_sampler = DistributedSampler(valid_dataset,
  #                                     num_replicas = xm.xrt_world_size(),
  #                                     rank = xm.get_ordinal(),
  #                                     shuffle = True)    
  ## loader
  train_loader = torch.utils.data.DataLoader(TextDataset(comment_text = train[num_proc*size_tr_dataset:(num_proc+1)*size_tr_dataset].comment_text.values,
                                                         targets=train[num_proc*size_tr_dataset:(num_proc+1)*size_tr_dataset].toxic.values, max_length=200, tokenizer=tokenizer),
                                             batch_size=FLAGS['batch_size'], shuffle=False,
                                             num_workers = FLAGS['num_workers'])
                                              # sampler=train_sampler)
  val_loader = torch.utils.data.DataLoader(TextDataset(comment_text=valid[num_proc*size_val_dataset:(num_proc+1)*size_val_dataset].comment_text.values,
                                                       targets=valid[num_proc*size_tr_dataset:(num_proc+1)*size_tr_dataset].toxic.values, max_length=200, tokenizer=tokenizer),
                                            batch_size=FLAGS['batch_size'], shuffle=False,
                                            num_workers = FLAGS['num_workers'])
                                          #  sampler=valid_sampler, 
  print('created loaders')
  device = xm.xla_device()
  print('created device')
  t_total = len(train_loader)//gradient_accumulation_steps*FLAGS['num_epochs']
  warmup_proportion = 0.01
  num_warmup_steps = t_total * warmup_proportion

  # net = CustomizedBert.from_pretrained('bert-base-multilingual-uncased', config=bert_config)
  model = mx.to(device)
  print('created_model')
  # loss_window = deque(maxlen = FLAGS['log_steps'])
  loss_fn = torch.nn.BCEWithLogitsLoss()
  optimizer = AdamW(model.parameters(), lr = lr)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)  # PyTorch scheduler
  best_valid_loss = float("Inf")
  for epoch in range(1, FLAGS['num_epochs'] + 1):
    # train_loss = train_loop_fn(train_loader,net,FLAGS,device,scheduler,optimizer)
    # valid_loss, valid_score = valid_loop_fn(valid_loader,model,len(validation),device)
    # elapsed_time = time.time() - start_time 
    # print('Epoch {}/{} \t loss={:.4f}\t val_loss={:.4f} \t train_score={:.4f}\t val_score={:.4f}\t time={:.2f}s'
    #     .format(epoch, FLAGS['num_epochs'], train_loss, valid_loss, valid_score , elapsed_time))
    # if best_valid_loss > valid_loss:
    #     best_valid_loss = valid_loss
    #     save_checkpoint(file_path + '/' + 'net.pt', model, best_valid_loss)
    # print("Start training epoch {}".format(epoch))
    xm.master_print("Start training epoch {}".format(epoch))
    start_time = time.time()
    t = time.time()
    para_loader = pl.ParallelLoader(train_loader, [device])
    print('первая пошла')
    train_loss, train_score = train_loop_fn(para_loader.per_device_loader(device),model,device)
    print('Training epoch end')
    xm.master_print(f'[RESULT]: Train. Epoch: {epoch}, loss: {train_loss.avg:.5f}, final_score: {train_score.avg:.5f}, time: {(time.time() - t):.5f}')
    print(f'[RESULT]: Train. Epoch: {epoch}, loss: {train_loss.avg:.5f}, final_score: {train_score.avg:.5f}, time: {(time.time() - t):.5f}')
    del para_loader

    t = time.time()
    para_loader = pl.ParallelLoader(valid_loader, [device])
    valid_loss, valid_score = valid_loop_fn(para_loader.per_device_loader(device),model,device)
    xm.master_print(f'[RESULT]: Validation. Epoch: {self.epoch}, loss: {losses.avg:.5f}, final_score: {final_scores.avg:.5f}, time: {(time.time() - t):.5f}')
    del para_loader

    print('Epoch {}/{} \t loss={:.4f}\t val_loss={:.4f} \t train_score={:.4f}\t val_score={:.4f}\t time={:.2f}s'
          .format(epoch, FLAGS['num_epochs'], train_loss.avg, valid_loss.avg, train_score.avg, valid_score.avg, (time.time() - start_time)))
    if best_valid_loss > valid_loss.avg:
        best_valid_loss = valid_loss.avg
        save_checkpoint('net.pt', model, best_valid_loss)
    del train_loss, valid_loss, train_score, valid_score

    # cb.put(net, val_loss)

In [ ]:
def _mp_fn(rank, flags):
    FLAGS = flags
    torch.set_default_tensor_type('torch.FloatTensor')
    train_start = time.time()
    train_net(flags)
    elapsed_train_time = time.time() - train_start
    print("Process", rank, "finished training. Train time was:", elapsed_train_time)

In [ ]:
FLAGS = {}
FLAGS['seed'] = 1
FLAGS['batch_size'] = 32
FLAGS['num_workers'] = 0
FLAGS['num_epochs'] = 2
FLAGS['num_cores'] = 8
FLAGS['log_steps'] = 20
xmp.spawn(_mp_fn, args = (FLAGS,), nprocs = FLAGS['num_cores'], start_method='fork')

created loaders
created device
created loaders
created device
created loaders
created device
created loaders
created device
created loaders
created loaders
created device
created device


ProcessExitedException: ignored

In [ ]:
device=xm.xla_device()
net.to(device)

ToxicSimpleNNModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
import re
import os
import torch
import pandas as pd
from scipy import stats
import numpy as np

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib

import logging
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule, BertModel
import sys
from sklearn import metrics, model_selection

import warnings
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import warnings

warnings.filterwarnings("ignore")

class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768 * 2, 1)

    def forward(
            self,
            ids,
            mask,
            token_type_ids
    ):
        output = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids)
        
        o1 = output[0]
        
        apool = torch.mean(o1, 1)
        mpool, _ = torch.max(o1, 1)
        cat = torch.cat((apool, mpool), 1)

        bo = self.bert_drop(cat)
        p2 = self.out(bo)
        return p2


class BERTDatasetTraining:
    def __init__(self, comment_text, targets, tokenizer, max_length):
        self.comment_text = comment_text
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.targets = targets

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, item):
        comment_text = str(self.comment_text[item])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        padding_length = self.max_length - len(ids)
        
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[item], dtype=torch.float)
        }

In [ ]:
class ToxicSimpleNNModel(nn.Module):

    def __init__(self):
        super(ToxicSimpleNNModel, self).__init__()
        self.encoder = BertModel.from_pretrained('bert-base-multilingual-uncased')
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(
            in_features=self.encoder.pooler.dense.out_features*2,
            out_features=1,
        )

    def forward(self, input_ids, attention_masks):
        # bs, seq_length = input_ids.shape
        seq_x = self.encoder(input_ids=input_ids, attention_mask=attention_masks)
        apool = torch.mean(seq_x[0], 1)
        mpool, _ = torch.max(seq_x[0], 1)
        x = torch.cat((apool, mpool), 1)
        x = self.dropout(x)
        return self.linear(x)

In [ ]:
def clean(text):
  if re.findall(r'\[\'\'\]',text):
    return ''
  else:
    retext = re.split(',',text)
    clean_text = []
    retext[0] = retext[0][1:]
    retext[-1] = retext[-1][:-1]
    for i in range(len(retext)):
        retext[i] = " ".join(retext[i].split())
        retext[i] = retext[i][1:-1]
    return '.'.join(retext)

def clean_test(text):
  if re.findall(r'\[\'\'\]',text):
    return '[UNK]'
  else:
    retext = re.split(',',text)
    clean_text = []
    retext[0] = retext[0][1:]
    retext[-1] = retext[-1][:-1]
    for i in range(len(retext)):
        retext[i] = " ".join(retext[i].split())
        retext[i] = retext[i][1:-1]
    return '.'.join(retext)

In [ ]:
mx = ToxicSimpleNNModel()
# df_train1 = pd.read_csv("/content/drive/My Drive/Epam/train_sent_prep.csv", usecols=["comment_text", "toxic"],converters={'comment_text': lambda x: clean(x)}).fillna("none")
# df_train1 = df_train1[df_train1.comment_text!='']
# df_train_full = df_train1.reset_index(drop=True)
# df_train = df_train_full.sample(frac=1).reset_index(drop=True)

# df_valid = pd.read_csv('/content/drive/My Drive/Epam/validation_sent_pr.csv', usecols=["comment_text", "toxic"], converters={'comment_text': lambda x: clean(x)})
# df_valid = df_valid[df_valid.comment_text!='']

In [ ]:
def train_loop_fn(loader,model,device):
  tracker = xm.RateTracker()
  # cb = callback()
  losses = AverageMeter()
  auc = RocAucMeter()
  # start_time = time.time()


  gradient_accumulation_steps = 1
  # avg_loss = 0.0
  model.train()
  for step, data in enumerate(tqdm(loader)):
      input_ids, input_masks, labels = data
      pred = model(input_ids = input_ids,
                    attention_masks = input_masks
                  )
      # train_preds.extend(torch.sigmoid(pred).cpu().detach().numpy())
      # train_labels.extend(labels.float())
      loss = loss_fn(pred, labels.float())
      # avg_loss += loss.item()
      # loss = loss / gradient_accumulation_steps
      loss.backward()
      loss = loss.detach().item()
        
      auc.update(labels, pred)
      losses.update(loss, input_ids.size(0))
      if (step + 1) % gradient_accumulation_steps == 0:
          # Calling the step function on an Optimizer makes an update to its parameters
          xm.optimizer_step(optimizer)
          scheduler.step()
          optimizer.zero_grad()

      tracker.add(FLAGS['batch_size'])
      if (step+1) % FLAGS['log_steps'] == 0:
          print('[xla:{}]({}) Rate={:.2f} Loss={:.4f} AUC={:.4f} GlobalRate={:.2f} Time={}'.format(
              xm.get_ordinal(), step+1, tracker.rate(), losses.avg, auc.avg,
              tracker.global_rate(), time.asctime()), flush=True)
  del loss
  del losses
  del outputs
  del input_ids
  del labels
  
  return losses, auc

def valid_loop_fn(loader,model,device): 
  fin_targets = []
  fin_outputs = []   
  losses = AverageMeter()
  final_scores = RocAucMeter()

  model.eval()
  # valid_preds = np.zeros((validation_length, 1))
  # true_label = np.zeros((validation_length, 1))
  with torch.no_grad():
    for j, data in enumerate(val_loader):

        # get the inputs
        input_ids, input_masks, labels = data
        pred = net(input_ids = input_ids.long(),
                          attention_mask = input_masks
                        )

        loss_val = loss_fn(pred, labels.float())
        final_scores.update(targets, outputs)
        losses.update(loss.detach().item(), input_ids.size(0))
        del loss_val,pred,input_ids,input_masks,labels

  return losses, final_scores

In [ ]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            input_ids=ids,
            attention_masks=mask
            # token_type_ids=token_type_ids
        )

        loss = loss_fn(outputs, targets)
        if bi % 10 == 0:
            xm.master_print(f'bi={bi}, loss={loss}')

        loss.backward()
        xm.optimizer_step(optimizer)
        if scheduler is not None:
            scheduler.step()

def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(
            input_ids=ids,
            attention_masks=mask
            # token_type_ids=token_type_ids
        )

        targets_np = targets.cpu().detach().numpy().tolist()
        outputs_np = outputs.cpu().detach().numpy().tolist()
        fin_targets.extend(targets_np)
        fin_outputs.extend(outputs_np)    

    return fin_outputs, fin_targets

In [ ]:
def _run():
    
    MAX_LEN = 192
    TRAIN_BATCH_SIZE = 64
    EPOCHS = 2

    tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-multilingual-uncased", do_lower_case=True)

    train_targets = df_train.toxic.values
    valid_targets = df_valid.toxic.values

    train_dataset = BERTDatasetTraining(
        comment_text=df_train.comment_text.values,
        targets=train_targets,
        tokenizer=tokenizer,
        max_length=MAX_LEN
    )

    train_sampler = torch.utils.data.distributed.DistributedSampler(
          train_dataset,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(),
          shuffle=True)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=4
    )

    valid_dataset = BERTDatasetTraining(
        comment_text=df_valid.comment_text.values,
        targets=valid_targets,
        tokenizer=tokenizer,
        max_length=MAX_LEN
    )

    valid_sampler = torch.utils.data.distributed.DistributedSampler(
          valid_dataset,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(),
          shuffle=False)

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=64,
        sampler=valid_sampler,
        drop_last=False,
        num_workers=4
    )

    device = xm.xla_device()
    model = mx.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    lr = 0.4 * 1e-5 * xm.xrt_world_size()
    num_train_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE / xm.xrt_world_size() * EPOCHS)
    xm.master_print(f'num_train_steps = {num_train_steps}, world_size={xm.xrt_world_size()}')

    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )

    for epoch in range(EPOCHS):
        para_loader = pl.ParallelLoader(train_data_loader, [device])
        train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler=scheduler)

        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)
        xm.save(model.state_dict(), "model.bin")
        auc = metrics.roc_auc_score(np.array(t) >= 0.5, o)
        xm.master_print(f'AUC = {auc}')

In [ ]:
def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    a = _run()
    
FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

num_train_steps = 1092, world_size=8


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

bi=0, loss=0.8381646871566772


KeyboardInterrupt: ignored

In [ ]:
import random as rand
otd = ['Алейник','Валиев','Дудин','Миронов','Пономарев','Севергин','Сереберенников']
rand.choices((otd),k=2)

['Алейник', 'Сереберенников']

In [2]:
import random as rand
otd = ['Дудин','Миронов','Пономарев','Севергин','Сереберенников','Сухоженко']
# rand.choices((otd),k=2)
rand.choice(otd)

'Сухоженко'